In [1]:
import numpy as np
import pandas as pd
import csv
import re
import matplotlib.pyplot as plt



In [2]:
#reading in data sets 
_1718 = pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2017-18_Data.txt", sep = '\t',dtype = {'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'pct_l1':object, 'pct_l2':object, 'pct_l3':object, 'pct_l4':object, 'pct_l5':object, 'pct_ccr':object, 'pct_glp':object, 'avg_score':float})     
_1617 = pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2016-17_Data.txt", sep = '\t',dtype = {'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
_1516 = pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2015-16_Data.txt", sep = '\t',dtype = {'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
_1415 = pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2014-15_Data.txt", sep = '\t',dtype = {'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
_1314 = pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2013-14_Data.txt", sep = '\t',dtype = {'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
school= pd.read_csv(r"C:\Users\Derek\Desktop\proj\school.txt", sep = '\t', dtype = {'school_code':str,'grade_span':object,'school_name':object})

In [3]:
#adding years to the dataset
_1718['year'] = 2017
_1617['year'] = 2016
_1516['year'] = 2015
_1415['year'] = 2014
_1314['year'] = 2013

In [4]:
#set all data on top of each other

data = _1718.append(_1617.append(_1516.append(_1415.append(_1314))))
data = data.replace(['*','<'],'0')
data.fillna(0 , inplace = True)
school.fillna(0 , inplace = True)

#dropped columns that were not useful

data = data.drop(['grade_span','avg_score'], axis=1)

In [5]:
#merged on school_code and made prefix numbers to merge on districts

data1 = pd.merge(data, school, on = ['school_code'])
data1['school_prefix_number'] = data1['school_code'].astype(str).str[:3]
data1['school_suffix_number'] = data1['school_code'].astype(str).str[3:6]
data1['school_prefix_number2'] = data1['school_prefix_number'].astype(str).str[:2]
data1.fillna('none', inplace = True)

#get rid of NC- prefixes, they are SBE districts that we don't need

data1 = data1[data1['school_prefix_number'] != 'NC-']

# data takes to long to run with all subgroups and types so we narrowed it down to just 'all'

data1 = data1[data1['subgroup'] == 'ALL']
data1 = data1[data1['type'] == 'ALL']
data1 = data1[data1['grade'] == 'ALL']

In [6]:
#reading in district data sets

public = pd.read_csv(r"C:\Users\Derek\Desktop\proj\public.txt", sep = '\t', dtype = {'school_prefix_number':object,'district_name':str,'phone':object})
charter = pd.read_csv(r"C:\Users\Derek\Desktop\proj\charter.csv", dtype = {'district_name':str,'school_prefix_number':object,'phone':object})
county = pd.read_csv(r"C:\Users\Derek\Desktop\proj\county.txt", sep = '\t',dtype = {'district_name':str,'City':object,'County name':object})

#appended charter and public school district datasets together

district = public.append(charter)

#create a way to make district names match county district names 

district['district_name'] = district['district_name'].str.replace(r"(Schools)",r"School District ")
district['district_name'] = district['district_name'].str.replace(r"(Academy)",r"Academy School District ")

# removed trailing spaces

district['district_name'] = district['district_name'].map(lambda x: x.strip())
county['district_name'] = county['district_name'].map(lambda x: x.strip())


In [7]:
#merged districts and county data on district names and created prefix2 for code for all distinct counties

district1 = pd.merge(district, county, on = ['district_name'], how = 'left')
district1['school_prefix_number2'] = district1['school_prefix_number'].astype(str).str[:2]


In [8]:
#create district 2 dataframe with the unique county school prefixes and the name of those counties.

district2 = district1
district2 = district2.drop(['phone','City ','school_prefix_number'], axis = 1)
district1 = district1.drop(['County name','district_name'], axis = 1)
district2 = district2.dropna(subset = ['County name'])
district2 = district2.drop_duplicates(subset = ['school_prefix_number2'])


In [9]:
#merging counties onto the districts for all districts, 

district3 = pd.merge(district1, district2, on = ['school_prefix_number2'])
district3.to_csv('district.csv', sep = ',', index = False)

In [10]:
#left merged data1 on to district to put each school into their county

data2 = pd.merge(data1,district3, on = ['school_prefix_number2'], how = 'left')
data2.loc[data2['school_prefix_number_x'] == '590', 'County name'] = 'McDowell'
data2.loc[data2['school_prefix_number_x'] == '995', 'County name'] = 'Yancey'
data2.loc[data2['school_prefix_number_x'] >= '996', 'County name'] = 'State'

data2.loc[data2['school_prefix_number_x'] == '590', 'district_name'] = 'McDowell County School District'
data2.loc[data2['school_prefix_number_x'] == '995', 'district_name'] = 'Yancey County School District'
data2.loc[data2['school_prefix_number_x'] >= '996', 'district_name'] = 'State School District'
data2['school_prefix_number'] = data2['school_prefix_number_x']
data2 = data2.drop(['grade_span', 'school_prefix_number_x', 'school_prefix_number_y', 'City ', 'phone', 'name', 'num_l1', 'num_l2', 'num_l3', 'num_l4', 'num_l5'], axis = 1)      

In [11]:
#seeing if there is any missing counties 

man = data2[data2['County name'] == '']
man['County name'].describe()

count     0
unique    0
Name: County name, dtype: int64

In [12]:
#got rid of Handicapped and prison schools

data2 = data2[data2['school_prefix_number'] != '298']
data2 = data2[data2['school_suffix_number'] != 'LEA']

#put in city districts

data2.loc[data2['school_prefix_number'] == '111', 'district_name'] = 'Asheville City School District'
data2.loc[data2['school_prefix_number'] == '132', 'district_name'] = 'Kannapolis City School District'
data2.loc[data2['school_prefix_number'] == '181', 'district_name'] = 'Hickory City School District'
data2.loc[data2['school_prefix_number'] == '182', 'district_name'] = 'Newton Conover City School District'
data2.loc[data2['school_prefix_number'] == '241', 'district_name'] = 'Whiteville City School District'
data2.loc[data2['school_prefix_number'] == '291', 'district_name'] = 'Lexington City School District'
data2.loc[data2['school_prefix_number'] == '292', 'district_name'] = 'Thomasville City School District'
data2.loc[data2['school_prefix_number'] == '421', 'district_name'] = 'Roanoke Rapids City School District'
data2.loc[data2['school_prefix_number'] == '422', 'district_name'] = 'Weldon City School District'
data2.loc[data2['school_prefix_number'] == '491', 'district_name'] = 'Mooresville Graded School District'
data2.loc[data2['school_prefix_number'] == '681', 'district_name'] = 'Chapel Hill-Carrboro City School District'
data2.loc[data2['school_prefix_number'] == '761', 'district_name'] = 'Asheboro City School District'
data2.loc[data2['school_prefix_number'] == '821', 'district_name'] = 'Clinton City School District'
data2.loc[data2['school_prefix_number'] == '861', 'district_name'] = 'Elkin City School District'
data2.loc[data2['school_prefix_number'] == '862', 'district_name'] = 'Mount Airy City School District'

data2 = data2.drop_duplicates(subset = ['year','school_name','pct_glp','num_tested','subject'])

In [13]:
#replaced less than 5 and greater than 95

data2 = data2.replace('<5','5')
data2 = data2.replace('>95','95')

#made columns numeric to use for math

data2['num_tested'] = data2['num_tested'].astype(int)
data2['pct_ccr'] = data2['pct_ccr'].astype(float)
data2['pct_glp'] = data2['pct_glp'].astype(float)
data2 = data2.drop(['pct_l1','pct_l2','pct_l3','pct_l4','pct_l5','num_ccr','num_glp','school_suffix_number','school_prefix_number2','school_prefix_number'],axis=1)      


In [14]:
# percent of on grade level
dat = data2.groupby('year')['pct_glp'].agg(['min','mean','max','std']).reset_index()
dat

,year,min,mean,max,std
0,2013,5.0,54.120283,95.0,17.274990
1,2014,5.0,54.606488,95.0,17.736141
2,2015,5.0,56.354075,95.0,17.697201
3,2016,5.0,57.258877,95.0,17.634389
4,2017,5.0,56.984096,95.0,17.597565


In [15]:
#number of 5% on grade level schools
data2[data2['pct_glp'] == data2['pct_glp'].min()].count()

grade            64
num_tested       64
pct_ccr          64
pct_glp          64
school_code      64
subgroup         64
subject          64
type             64
year             64
school_name      64
district_name    64
County name      64
dtype: int64

In [16]:
# percent of above grade level
dat = data2.groupby('year')['pct_ccr'].agg(['min','mean','max','std']).reset_index()
dat

,year,min,mean,max,std
0,2013,5.0,43.694141,95.0,17.101249
1,2014,5.0,44.469911,95.0,17.576643
2,2015,5.0,46.493358,95.0,17.683005
3,2016,5.0,46.766298,95.0,17.698553
4,2017,5.0,47.006963,95.0,17.667830


In [17]:
# getting correct years and subjects to merge onto data2

size =pd.read_csv(r"C:\Users\Derek\Desktop\proj\ClassSize.csv")
size = size[size['year'] >= 2013]
size = size[size['year'] <= 2017]
size = size[size['level'] != 'Biology']
size = size[size['level'] != 'English II']
size = size[size['level'] != 'Math I']
size['school_code']=size['unit_code'].astype(object)


In [18]:
#grouped by columns 

size2 = size.groupby(['year','school_code','School_Name','State_Name','Lea_Name'])[['size','lea_size','st_size']].mean()
size2 = size2.reset_index()

In [19]:
#merged size

new = pd.merge(data2,size2, on=['school_code','year'], how = 'inner')
new['pct_glp']=new['pct_glp'].astype(float)
new.head()

,grade,num_tested,pct_ccr,pct_glp,school_code,subgroup,subject,type,year,school_name,district_name,County name,School_Name,State_Name,Lea_Name,size,lea_size,st_size
0,ALL,705,49.6,60.1,010304,ALL,ALL,ALL,2017,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,22.166667,20.5,19.666667
1,ALL,703,53.2,62.9,010304,ALL,ALL,ALL,2016,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,21.000000,19.5,19.500000
2,ALL,646,49.4,59.6,010304,ALL,ALL,ALL,2015,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,20.666667,20.0,19.333333
3,ALL,623,51.4,62.6,010304,ALL,ALL,ALL,2013,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,20.333333,19.5,20.166667
4,ALL,814,50.4,60.4,010308,ALL,ALL,ALL,2017,Altamahaw Ossipee Elem,Alamance-Burlington School District,Alamance,Altamahaw Ossipee Elem,State Of North Carolina,Alamance-Burlington Schools,21.833333,20.5,19.666667


In [20]:
#create funds frm data

funds = pd.read_csv(r"C:\Users\Derek\Desktop\proj\funds.csv")
funds = funds[funds['year'] >= 2013]
funds = funds[funds['year'] <= 2017]
funds['school_code']=funds['unit_code'].astype(object)
funds.head()

,year,unit_code,Lea_Name,School_Name,State_Name,category_cd,total_expense_num,salary_expense_pct,benefits_expense_pct,services_expense_pct,...,st_supplies_expense_pct,st_instruct_equip_exp_pct,st_other_expense_pct,st_federal_perpupil_num,st_local_perpupil_num,st_state_perpupil_num,building_expense_pct,lea_building_expense_pct,st_building_expense_pct,school_code
27629,2013,291340,Lexington City Schools,Pickett Elementary School,State Of North Carolina,E,NaN,NaN,NaN,NaN,...,0.086,0.006,0.0,1007.57,2094.72,5395.31,NaN,NaN,NaN,291340
27630,2013,130333,Cabarrus County Schools,"Performance Learning Center, Concord",State Of North Carolina,H,NaN,NaN,NaN,NaN,...,0.086,0.006,0.0,1007.57,2094.72,5395.31,NaN,NaN,NaN,130333
27631,2013,130318,Cabarrus County Schools,C C Griffin Middle,State Of North Carolina,M,NaN,NaN,NaN,NaN,...,0.086,0.006,0.0,1007.57,2094.72,5395.31,NaN,NaN,NaN,130318
27632,2013,130316,Cabarrus County Schools,Jay M Robinson High,State Of North Carolina,H,NaN,NaN,NaN,NaN,...,0.086,0.006,0.0,1007.57,2094.72,5395.31,NaN,NaN,NaN,130316
27633,2013,130322,Cabarrus County Schools,"Mount Pleasant Elementary, Mount Pleasant",State Of North Carolina,E,NaN,NaN,NaN,NaN,...,0.086,0.006,0.0,1007.57,2094.72,5395.31,NaN,NaN,NaN,130322


In [21]:
# merge funds

new2 = pd.merge(new,funds, on = ['year','school_code','Lea_Name','School_Name','State_Name'], how = 'inner')
new2.head()

,grade,num_tested,pct_ccr,pct_glp,school_code,subgroup,subject,type,year,school_name,...,st_services_expense_pct,st_supplies_expense_pct,st_instruct_equip_exp_pct,st_other_expense_pct,st_federal_perpupil_num,st_local_perpupil_num,st_state_perpupil_num,building_expense_pct,lea_building_expense_pct,st_building_expense_pct
0,ALL,705,49.6,60.1,010304,ALL,ALL,ALL,2017,Alexander Wilson Elementary,...,0.091,0.075,0.012,0.0,973.02,2231.65,5944.43,NaN,NaN,NaN
1,ALL,703,53.2,62.9,010304,ALL,ALL,ALL,2016,Alexander Wilson Elementary,...,0.090,0.075,0.010,0.0,967.75,2172.86,5718.43,NaN,NaN,NaN
2,ALL,646,49.4,59.6,010304,ALL,ALL,ALL,2015,Alexander Wilson Elementary,...,0.089,0.079,0.009,0.0,978.59,2144.66,5628.91,NaN,NaN,NaN
3,ALL,623,51.4,62.6,010304,ALL,ALL,ALL,2013,Alexander Wilson Elementary,...,0.084,0.086,0.006,0.0,1007.57,2094.72,5395.31,NaN,NaN,NaN
4,ALL,814,50.4,60.4,010308,ALL,ALL,ALL,2017,Altamahaw Ossipee Elem,...,0.091,0.075,0.012,0.0,973.02,2231.65,5944.43,NaN,NaN,NaN


In [22]:
#get rid of missing lea_size

size = new[new['lea_size']>=0]
size.head()

,grade,num_tested,pct_ccr,pct_glp,school_code,subgroup,subject,type,year,school_name,district_name,County name,School_Name,State_Name,Lea_Name,size,lea_size,st_size
0,ALL,705,49.6,60.1,010304,ALL,ALL,ALL,2017,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,22.166667,20.5,19.666667
1,ALL,703,53.2,62.9,010304,ALL,ALL,ALL,2016,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,21.000000,19.5,19.500000
2,ALL,646,49.4,59.6,010304,ALL,ALL,ALL,2015,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,20.666667,20.0,19.333333
3,ALL,623,51.4,62.6,010304,ALL,ALL,ALL,2013,Alexander Wilson Elementary,Alamance-Burlington School District,Alamance,Alexander Wilson Elementary,State Of North Carolina,Alamance-Burlington Schools,20.333333,19.5,20.166667
4,ALL,814,50.4,60.4,010308,ALL,ALL,ALL,2017,Altamahaw Ossipee Elem,Alamance-Burlington School District,Alamance,Altamahaw Ossipee Elem,State Of North Carolina,Alamance-Burlington Schools,21.833333,20.5,19.666667


In [23]:
#exported

new.to_csv('sizeANDfunds.csv', sep = ',', index=False)

In [24]:
#correlation of size
new2[['pct_glp','size']].corr()

,pct_glp,size
pct_glp,1.000000,0.166665
size,0.166665,1.000000


In [25]:
#highest class size school

new[new['size'] == new['size'].max()]

,grade,num_tested,pct_ccr,pct_glp,school_code,subgroup,subject,type,year,school_name,district_name,County name,School_Name,State_Name,Lea_Name,size,lea_size,st_size
5242,ALL,1579,44.6,60.9,66A000,ALL,ALL,ALL,2017,Gaston College Preparatory,Northampton County School District,Northampton,Gaston College Preparatory,State Of North Carolina,Charter and Non-District Affiliated Schools,55.384615,NaN,20.153846


In [26]:
#looking at class size

new2['size'].describe()

count    8070.000000
mean       20.185965
std         3.250463
min        10.000000
25%        18.333333
50%        20.166667
75%        22.000000
max        55.384615
Name: size, dtype: float64

In [27]:
#confidence interval of class size

import math
rootN=math.sqrt(8070)
s=3.250463
z=1.96
x=20.185965
y=x+z*s
t=x-z*s
print("Confidence Interval of Average Class Size is [", t,',',y,"]")

Confidence Interval of Average Class Size is [ 13.81505752 , 26.55687248 ]


In [28]:
#correlation of funds

new2[['pct_glp','total_expense_num','salary_expense_pct','benefits_expense_pct','supplies_expense_pct']].corr()

,pct_glp,total_expense_num,salary_expense_pct,benefits_expense_pct,supplies_expense_pct
pct_glp,1.000000,-0.390997,0.217619,0.120706,-0.284929
total_expense_num,-0.390997,1.000000,-0.162648,-0.172573,0.113061
salary_expense_pct,0.217619,-0.162648,1.000000,0.414803,-0.128709
benefits_expense_pct,0.120706,-0.172573,0.414803,1.000000,-0.126414
supplies_expense_pct,-0.284929,0.113061,-0.128709,-0.126414,1.000000
